In [55]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório
import os
import pathlib
import glob
from IPython.display import clear_output

#Para Regressão Linear Simples
from scipy import stats

#Para Regressão Linear Múltipla
import statsmodels.api as sm

##Criando as Funções
def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata
    O arquivo deve estar na mesma pasta do arquivo de Python
    Deixe em branco para ler o arquivo mais recentemente adicionado
    '''
    global df
    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
        df = pd.read_stata(arquivo)
        print(f"{arquivo}.dta foi lido com sucesso!")
        return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except: #caso não tenha sido encontrado o arquivo com o nome inserido
            print('''
            Não foi possível achar o arquivo :(\n
            Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
            ''')

def Regressão_Simples(Lista_x, Lista_y):
    '''
    Função que calcula as estatísticas de uma regressão simples
    '''
    global Lista_ychapeu_simples, Resíduos_simples
    
    #calculando o número de observações e as médias
    Número_de_Observações = len(Lista_x)
    Média_x = sum(Lista_x)/Número_de_Observações
    Média_y = sum(Lista_y)/Número_de_Observações

    #Calculando os coeficientes do modelo
    B1, B0, R, valor_P, DP = stats.linregress(Lista_x, Lista_y)
    #Gerando os valores previstos
    Lista_ychapeu_simples = [round(B0 + B1 * i,3) for i in Lista_x]
    #Calculando os Resíduos
    Resíduos_simples = [(j - k) for j,k in zip(Lista_y, Lista_ychapeu_simples)]
    
    #Calculando R-quadrados e a Soma dos Quadrados das Variáveis
    R_quadrado = round(R**2,5)
    SQTx = sum([(i - Média_x)**2 for i in Lista_x])
    SQTy = sum([(i - Média_y)**2 for i in Lista_y])
    SQEy = sum([(i - Média_y)**2 for i in Lista_ychapeu_simples])
    SQR = sum([i**2 for i in Resíduos_simples]) 

    #Calculando a Variância da Regressão e dos  Coeficientes
    VarianciaReg = SQR/(Número_de_Observações - 2)
    EPR = math.sqrt(VarianciaReg)
    VarB1 = VarianciaReg/SQTx
    VarB0 = (VarianciaReg * sum([i**2 for i in Lista_x]))/(Número_de_Observações * SQTx)
    
    #Calculando da estatistica t com intervalo de confiança de 95%  (p/ gerar os intervalos de confiança dos estimadores)
    Estatistica_t_Critica = stats.t.ppf(0.95, Número_de_Observações - 2)

    #Calculando os estimadores do limite inferior e superior
    B1_inferior = B1 - math.sqrt(VarB1) * Estatistica_t_Critica
    B1_superior = B1 + math.sqrt(VarB1) * Estatistica_t_Critica
    B0_inferior = B0 - math.sqrt(VarB0) * Estatistica_t_Critica
    B0_superior = B0 + math.sqrt(VarB0) * Estatistica_t_Critica
    
    #Gerando o Relatório
    Relatório = f'''
    Número de Observações = {Número_de_Observações}\n
    B0 = {round(B0,5)}\t B1 = {round(B1,5)}\t R-quadrado = {R_quadrado}\n
    Estimador da Variância = {round(VarianciaReg,5)}\t Erro Padrão da Regressão = {round(EPR,5)}\n
    Variância de B1 = {round(VarB1,5)}\t Variância de B0 = {round(VarB0,5)}\n
    Intervalo de Confiança de 95% para B1 (Inferior; B1; Superior): {round(B1_inferior,4)}; {round(B1,4)}; {round(B1_superior,4)}\n
    Intervalo de Confiança de 95% para B0 (Inferior; B0; Superior): {round(B0_inferior,4)}; {round(B0,4)}; {round(B0_superior,4)}\n
    Para ver os valores previstos, basta chamar a variável 'Lista_ychapeu_simples'\n
    Para ver os resíduos, chame a variável 'Resíduos_simples'
    '''
    print (Relatório)
    
    ##Criando um gráfico
    #Deixando o estilo bonitinho
    sns.set_style(style="white")

    #Criando o objeto gráfico
    Grafico = sns.regplot(x = Lista_x, y = Lista_y, scatter_kws={"color": "black"}, line_kws={"color": "red"})
    Grafico.set_title("Resultado da Regressão",fontsize = 11)
    plt.show()

def Regressão_Múltipla(x,y):
    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR
    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    X = sm.add_constant(x)
    #Criando o Modelo
    Modelo = sm.OLS(y,X)
    Resultado = Modelo.fit()
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(SQR/GL)
    
    ##Printando o Resultado
    #print('Parâmetros:\n', Resultado.params) #O primeiro resultado equivale ao intercepto
    #print('\nDesvios Padrões:\n', Resultado.bse)
    #print('Valores Previstos: ', Resultado.predict())
    #print('\nR2:', Resultado.rsquared)
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
  
def Teste_F_Restrições_de_Exclusão(x, y, Restrições, Nivel_de_Significância = 0.05):
    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições =list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressão_Múltipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressão_Múltipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    

## Testes de Restrições de Exclusão

In [22]:
coletar_dados('MLB1')

MLB1.dta foi lido com sucesso!


,salary,teamsal,nl,years,games,atbats,runs,hits,doubles,triples,...,runsyr,percwhte,percblck,perchisp,blckpb,hispph,whtepw,blckph,hisppb,lsalary
0,6329213.0,38407380.0,1,12,1705,6705,1076,1939,320,67,...,89.666664,70.277969,18.844229,10.877804,0.000000,0.000000,70.277969,0.000000,0.000000,15.660686
1,3375000.0,38407380.0,1,8,918,3333,407,863,156,38,...,50.875000,70.277969,18.844229,10.877804,18.844229,0.000000,0.000000,10.877804,0.000000,15.031906
2,3100000.0,38407380.0,1,5,751,2807,370,840,148,18,...,74.000000,70.277969,18.844229,10.877804,0.000000,0.000000,70.277969,0.000000,0.000000,14.946913
3,2900000.0,38407380.0,1,8,1056,3337,405,816,143,18,...,50.625000,70.277969,18.844229,10.877804,0.000000,0.000000,70.277969,0.000000,0.000000,14.880221
4,1650000.0,38407380.0,1,12,1196,3603,437,928,19,16,...,36.416668,70.277969,18.844229,10.877804,18.844229,0.000000,0.000000,10.877804,0.000000,14.316286
5,700000.0,38407380.0,1,17,2032,7489,1136,2145,270,142,...,66.823532,70.277969,18.844229,10.877804,18.844229,0.000000,0.000000,10.877804,0.000000,13.458836
6,695000.0,38407380.0,1,4,394,1293,179,303,51,13,...,44.750000,70.277969,18.844229,10.877804,0.000000,10.877804,0.000000,0.000000,18.844229,13.451667
7,275000.0,38407380.0,1,10,432,1005,78,240,35,5,...,7.800000,70.277969,18.844229,10.877804,0.000000,0.000000,70.277969,0.000000,0.000000,12.524527
8,235000.0,38407380.0,1,4,223,491,37,118,16,5,...,9.250000,70.277969,18.844229,10.877804,0.000000,10.877804,0.000000,0.000000,18.844229,12.367341
9,212500.0,38407380.0,1,3,156,434,45,116,16,0,...,15.000000,70.277969,18.844229,10.877804,18.844229,0.000000,0.000000,10.877804,0.000000,12.266697


In [23]:
#Calculando o modelo irrestrito
Regressão_Múltipla(df[['gamesyr','years','bavg','hrunsyr','rbisyr']],df['lsalary'])

O erro padrão da regressão é 0.7265772410070378 e a SQR é 183.18632704084152

                            OLS Regression Results                            
Dep. Variable:                lsalary   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     117.1
Date:                Fri, 02 Oct 2020   Prob (F-statistic):           2.94e-72
Time:                        11:28:28   Log-Likelihood:                -385.11
No. Observations:                 353   AIC:                             782.2
Df Residuals:                     347   BIC:                             805.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P&gt;|t|      [0.025      0.975]
--------------------------------------------------

In [24]:
#Calculando o modelo restrito
Regressão_Múltipla(df[['gamesyr','years']],df['lsalary'])

O erro padrão da regressão é 0.7527311744227924 e a SQR é 198.31147733177067

                            OLS Regression Results                            
Dep. Variable:                lsalary   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     259.3
Date:                Fri, 02 Oct 2020   Prob (F-statistic):           8.22e-70
Time:                        11:28:29   Log-Likelihood:                -399.11
No. Observations:                 353   AIC:                             804.2
Df Residuals:                     350   BIC:                             815.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P&gt;|t|      [0.025      0.975]
--------------------------------------------------

In [59]:
#Teste F
Teste_F_Restrições_de_Exclusão(df[['gamesyr','years','bavg','hrunsyr','rbisyr']],df['lsalary'],df[['bavg','hrunsyr','rbisyr']],0.1)

O valor de F é 9.55 e seu p-valor é 4.5e-06. Portanto, rejeita-se Ho a um nível de significância de 10.0%.


## Testes de Restrições Lineares Gerais

In [17]:
coletar_dados('hprice1')

hprice1.dta foi lido com sucesso!


,price,assess,bdrms,lotsize,sqrft,colonial,lprice,lassess,llotsize,lsqrft
0,300.000000,349.100006,4,6126.0,2438,1,5.703783,5.855359,8.720297,7.798934
1,370.000000,351.500000,3,9903.0,2076,1,5.913503,5.862210,9.200593,7.638198
2,191.000000,217.699997,3,5200.0,1374,0,5.252274,5.383118,8.556414,7.225482
3,195.000000,231.800003,3,4600.0,1448,1,5.273000,5.445875,8.433811,7.277938
4,373.000000,319.100006,4,6095.0,2514,1,5.921578,5.765504,8.715224,7.829630
5,466.274994,414.500000,5,8566.0,2754,1,6.144775,6.027073,9.055556,7.920810
6,332.500000,367.799988,3,9000.0,2067,1,5.806640,5.907539,9.104980,7.633853
7,315.000000,300.200012,3,6210.0,1731,1,5.752573,5.704449,8.733916,7.456455
8,206.000000,236.100006,3,6000.0,1767,0,5.327876,5.464255,8.699514,7.477038
9,240.000000,256.299988,3,2892.0,1890,0,5.480639,5.546349,7.969704,7.544332


In [18]:
Regressão_Múltipla(df[['lassess','llotsize','lsqrft','bdrms']], df['lprice'])

O erro padrão da regressão é 0.14814226284179982 e a SQR é 1.8215287933107793

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     70.58
Date:                Fri, 02 Oct 2020   Prob (F-statistic):           6.45e-26
Time:                        11:06:31   Log-Likelihood:                 45.750
No. Observations:                  88   AIC:                            -81.50
Df Residuals:                      83   BIC:                            -69.11
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P&gt;|t|      [0.025      0.975]
-------------------------------------------------

In [19]:
df['logpreco_logaval'] = df['lprice'] - df['lassess']

In [21]:
Regressão_Múltipla(df[[]], df['logpreco_logaval'])

O erro padrão da regressão é 0.1470064841014369 e a SQR é 1.880148854004344

                            OLS Regression Results                            
Dep. Variable:       logpreco_logaval   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 02 Oct 2020   Prob (F-statistic):                nan
Time:                        11:10:56   Log-Likelihood:                 44.357
No. Observations:                  88   AIC:                            -86.71
Df Residuals:                      87   BIC:                            -84.24
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P&gt;|t|      [0.025      0.975]
---------------------------------------------------

In [42]:
F = (1.88 - 1.822)/(4*0.148**2)
print(F)

0.6619795471146804
